In [1]:
# Import Required Libraries

import pandas as pd
import math
import numpy as np

import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.offline import iplot

from prophet import Prophet
import holidays
from prophet.diagnostics import cross_validation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

import tensorflow as tf
import os

from sklearn.preprocessing import MinMaxScaler
import joblib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

C:\Users\Arcega\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
def is_weekend(ds):
    date = pd.to_datetime(ds)
    # Return True for Saturday (5) and Sunday (6), False otherwise
    return date.weekday() >= 5

def df_to_X_y(df, window_size=6):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size][6]  # 'Entry' is the 7th column (index 6)
        y.append(label)
    return np.array(X), np.array(y)

def mean_absolute_scaled_error(test, y_true, y_pred):
    seasonality = 20

    seasonal_naive = test.copy()
    seasonal_naive['yhat'] = test['y'].shift(20)
    seasonal_naive['yhat'].fillna(test['y'][:20].mean(), inplace=True)

    naive_errors = np.abs(seasonal_naive['y'] - seasonal_naive['yhat'])

    # Mean absolute error of the naive forecast
    naive_mae = np.mean(naive_errors)
    naive_mae

    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the absolute error of the model's predictions
    abs_error = np.mean(np.abs(y_true - y_pred))
    
    # MASE calculation
    mase = abs_error / naive_mae

    return mase

def evaluate_model(test, test_forecast):
    # Evaluate performance
    mse = mean_squared_error(y_true=test['y'],  y_pred=test_forecast['yhat'])
    rmse = np.sqrt(mean_squared_error(y_true=test['y'], y_pred=test_forecast['yhat']))
    mae = mean_absolute_error(y_true=test['y'], y_pred=test_forecast['yhat'])
    r2 = r2_score(y_true=test['y'], y_pred=test_forecast['yhat'])
    mase = mean_absolute_scaled_error(test, y_true=test['y'],  y_pred=test_forecast['yhat'])
    return mse, rmse, mae, r2, mase


def prophet_model(train, test, df):
    # Create holidays dataframe
    holiday = pd.DataFrame([])
    for date, name in sorted(holidays.Philippines(years=[2022, 2023]).items()):
        holiday = pd.concat([holiday, pd.DataFrame({'ds': date, 'holiday': name}, index=[0])], ignore_index=True)
    holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')
    holiday.loc[holiday['ds'] == '2023-11-30', 'ds'] = '2023-11-27'

    # Initialize the Prophet model
    m = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        weekly_seasonality=True,
        holidays=holiday,
        seasonality_mode='multiplicative',
        seasonality_prior_scale=0.1,
        changepoint_prior_scale=0.001,

    )

    # Add custom seasonality
    m.add_seasonality(name='daily_is_weekend', period=1, fourier_order=4, condition_name='weekend')
    m.add_seasonality(name='daily_is_weekday', period=1, fourier_order=4, condition_name='weekday')

    # Add additional regressor
    m.add_regressor('off_hour')
    m.add_regressor('rain_amount')

    # Fit the model on the training data
    m.fit(train)

    # Make predictions for Test set
    test_forecast = m.predict(test)

    # Remove negative forecasts
    test_forecast['yhat'] = test_forecast['yhat'].apply(lambda x: max(x, 0))
    test_forecast['yhat_lower'] = test_forecast['yhat_lower'].apply(lambda x: max(x, 0))
    test_forecast['yhat_upper'] = test_forecast['yhat_upper'].apply(lambda x: max(x, 0))
    test_forecast['yhat'] = test_forecast['yhat'].round()

    # Round forecast values
    test_forecast['yhat'] = test_forecast['yhat'].round()

    # Create a future DataFrame with hourly intervals for the desired forecast period
    future = m.make_future_dataframe(periods=150, freq='D')
    future['hour'] = pd.to_datetime(future['ds']).dt.hour
    future['is_weekend'] = df['is_weekend']
    future['weekday'] = future['ds'].apply(is_weekend)
    future['weekend'] = ~future['ds'].apply(is_weekend)
    future['rain_amount'] = df['rain_amount']
    future['off_hour'] = future['hour'].apply(lambda x: 1 if (x >= 23) or (x <= 3) else 0)
    future = future[future['ds'].dt.hour < 23]
    future = future[future['ds'].dt.hour > 3]

    # make predictions
    forecast = m.predict(future)
    
    return test_forecast

def lstm_model(df, X_train, y_train, X_test, y_test, X_val, y_val):
    model = Sequential()
    model.add(InputLayer((X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(1, activation='linear'))
    model.summary()

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Define the ModelCheckpoint callback with the correct file path
    os.makedirs('model', exist_ok=True)
    cp1 = ModelCheckpoint(filepath='model/best_lstm.keras', save_best_only=True, monitor='val_loss', mode='min')

    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[cp1, early_stopping])

    # Assuming df is the original DataFrame with the 'Date Time' column
    # Extract the 'Date Time' column for the entire dataset
    date_time_test = df['Date Time']

    # Load the scaler for inverse transformation
    scaler_entry = joblib.load('model/scaler_entry.pkl')

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and actual values
    # Create a DataFrame to hold the predictions and actual values
    df_pred = pd.DataFrame(y_pred, columns=['Entry'])
    df_actual = pd.DataFrame(y_test, columns=['Entry'])

    # Inverse transform the 'Entry' column
    y_pred_inv = scaler_entry.inverse_transform(df_pred)
    y_test_inv = scaler_entry.inverse_transform(df_actual)

    # Ensure date_time_test matches the length of y_test_inv and y_pred_inv
    date_time_test = date_time_test[-len(y_test_inv):]

    # Round the predictions to the nearest whole number and ensure non-negative values
    y_pred_inv = np.round(np.maximum(y_pred_inv, 0))
    y_test_inv = np.round(np.maximum(y_test_inv, 0))

    # Calculate evaluation metrics
    l_mse = mean_squared_error(y_test_inv, y_pred_inv)
    l_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    l_mae = mean_absolute_error(y_test_inv, y_pred_inv)
    l_r2 = r2_score(y_test_inv, y_pred_inv)

    return y_pred_inv



In [3]:
# Import dataset
df = pd.read_csv('data/2223TaftProphet.csv', parse_dates=[0])

# Rename header to Prophet's requirements
df.reset_index()
df = df.rename(columns={'Datetime':'ds', 'Entry':'y'})

# Add additional regressors as columns in the dataframe
df['hour'] = pd.to_datetime(df['ds']).dt.hour
df['off_hour'] = df['hour'].apply(lambda x: 1 if (x >= 23) or (x <= 3) else 0)
df['weekday'] = ~df['ds'].apply(is_weekend)
df['weekend'] = df['ds'].apply(is_weekend)

# Split the dataset to training and testing sets
train_len = math.floor((df.shape[0]*80)/100)
train = df[:train_len]
test = df[train_len:]
test_len = math.floor((test.shape[0]*50)/100)
val = test[:test_len]
test = test[test_len:]

test_forecast = prophet_model(train, test, df)
prophet_eval = evaluate_model(test, test_forecast)
print(prophet_eval)


01:05:36 - cmdstanpy - INFO - Chain [1] start processing
01:06:10 - cmdstanpy - INFO - Chain [1] done processing


(446872.9226027397, 668.4855440491887, 516.9554794520548, 0.7565910827339437, 1.2373866501497068)


C:\Users\Arcega\AppData\Local\Temp\ipykernel_6040\3957576142.py:22: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.



In [4]:
# Load your time series data
df2 = pd.read_csv('data/2223TaftLSTM.csv')

# Set if holiday
ph_holidays = holidays.PH()
df2['is_holiday'] = df2['Date'].apply(lambda x: 1 if x in ph_holidays else 0)
df2.head(5)

# Combine 'Date' and 'Time' into a new column 'DateAndTime'
df2['DateAndTime'] = pd.to_datetime(df2['Date'] + ' ' + df2['Time'])

# Drop unneeded columns
df2.drop(['Date', 'Time', 'rain_amount', 'rain_desc'], axis=1, inplace=True)

 # Define additional features
df2['Date Time'] = pd.to_datetime(df2['DateAndTime'], format='%d.%m.%Y %H.%M.%S')
df2['hour'] = df2['Date Time'].dt.hour
df2['day_of_week'] = df2['Date Time'].dt.dayofweek
df2['month'] = df2['Date Time'].dt.month
df2['year'] = df2['Date Time'].dt.year
df2.drop(['DateAndTime', 'Day', 'is_weekend'], axis=1, inplace=True)
df2['is_weekend'] = df2['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
model_features = ['hour', 'day_of_week', 'is_weekend', 'month', 'year', 'rain_class', 'Entry']
df_model = df2[model_features]

# Normalize all features except 'Entry'
scaler = MinMaxScaler(feature_range=(0, 1))
df_model[df_model.columns[:-1]] = scaler.fit_transform(df_model[df_model.columns[:-1]])

# Save the scaler for the features
joblib.dump(scaler, 'model/scaler_features.pkl')

# Normalize the 'Entry' column separately
scaler_entry = MinMaxScaler(feature_range=(0, 1))
df_model['Entry'] = scaler_entry.fit_transform(df_model[['Entry']])

# Save the scaler for the 'Entry' column
joblib.dump(scaler_entry, 'model/scaler_entry.pkl')

#
X, y = df_to_X_y(df_model)

# Determine the split points
train_split_point = int(len(X) * 0.8)
valntest_split_point = int(len(X) * 0.9)

# Split the data
X_train, X_val, X_test = X[:train_split_point], X[train_split_point:valntest_split_point], X[valntest_split_point:]
y_train, y_val, y_test = y[:train_split_point], y[train_split_point:valntest_split_point], y[valntest_split_point:]

lstm_forecast = lstm_model(df2, X_train, y_train, X_test, y_test, X_val, y_val)

C:\Users\Arcega\AppData\Local\Temp\ipykernel_6040\3982592567.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Arcega\AppData\Local\Temp\ipykernel_6040\3982592567.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 64)             18432     
                                                                 
 dropout (Dropout)           (None, 6, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 53,569
Trainable params: 53,569
Non-traina

In [5]:
lstm_forecast = pd.DataFrame(lstm_forecast, columns=['yhat'])
lstm_forecast


,yhat
0,3.0
1,2583.0
2,4857.0
3,5612.0
4,4354.0
...,...
1455,1880.0
1456,1283.0
1457,37.0
1458,29.0


In [14]:
prophet_forecast = test_forecast.copy()

In [ ]:
# combine prophet and lstm forecast into one dataframe
test_actual = test

hybrid_forecast = prophet_forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet'})
hybrid_forecast['lstm'] = lstm_forecast['yhat']
hybrid_forecast['y'] = test_actual['y']
print(hybrid_forecast)

                      ds  prophet    lstm     y
0    2023-10-20 04:00:00      0.0     3.0     0
1    2023-10-20 05:00:00   2043.0  2583.0  2894
2    2023-10-20 06:00:00   3624.0  4857.0  4339
3    2023-10-20 07:00:00   4063.0  5612.0  5548
4    2023-10-20 08:00:00   3637.0  4354.0  4502
...                  ...      ...     ...   ...
1455 2023-12-31 19:00:00   2346.0  1880.0  1681
1456 2023-12-31 20:00:00   1828.0  1283.0   297
1457 2023-12-31 21:00:00    923.0    37.0     0
1458 2023-12-31 22:00:00      0.0    29.0     0
1459 2023-12-31 23:00:00      0.0     0.0     0

[1460 rows x 4 columns]


In [42]:
#Evaluate prophet
mse, rmse, mae, r2, mase = evaluate_model(hybrid_forecast,prophet_forecast)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")
print(f"MASE: {mase:.4f}")

MSE: 446872.9226
RMSE: 668.4855
MAE: 516.9555
R²: 0.7566
MASE: 1.2374


In [43]:
#Evaluate lstm
mse, rmse, mae, r2, mase = evaluate_model(hybrid_forecast,lstm_forecast)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")
print(f"MASE: {mase:.4f}")

MSE: 117929.3199
RMSE: 343.4084
MAE: 215.4418
R²: 0.9358
MASE: 0.5157


In [44]:
# Define weights
weight_prophet =0.1
weight_lstm = 0.9
# Calculate the hybrid forecast
hybrid_forecast['yhat'] = (
    weight_prophet * hybrid_forecast['prophet'] + 
    weight_lstm * hybrid_forecast['lstm']
)

actual = hybrid_forecast['y']
forecast = hybrid_forecast['yhat']

#Evaluate hybrid forecast
mse, rmse, mae, r2, mase = evaluate_model(hybrid_forecast, hybrid_forecast)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")
print(f"MASE: {mase:.4f}")

MSE: 120398.4172
RMSE: 346.9848
MAE: 222.7252
R²: 0.9344
MASE: 0.5331


In [45]:
def find_best_weights(hybrid_forecast, actuals_col, prophet_col, lstm_col, metric, step):
    best_metric = float('inf')
    best_weights = (0, 0)

    # Iterate through Prophet weights from 0 to 1, LSTM weights are (1 - prophet_weight)
    for w_prophet in np.arange(0, 1 + step, step):
        w_lstm = 1 - w_prophet
        
        # Calculate weighted forecast
        hybrid_forecast['hybrid'] = (w_prophet * hybrid_forecast[prophet_col]) + (w_lstm * hybrid_forecast[lstm_col])
        
        # Evaluate the hybrid forecast
        if metric == 'rmse':
            metric_value = np.sqrt(mean_squared_error(hybrid_forecast[actuals_col], hybrid_forecast['hybrid']))
        elif metric == 'mae':
            metric_value = np.mean(np.abs(hybrid_forecast[actuals_col] - hybrid_forecast['hybrid']))
        elif metric == 'mase':
            naive_forecast = hybrid_forecast[actuals_col].shift(20).dropna()
            naive_errors = np.abs(hybrid_forecast[actuals_col].iloc[20:] - naive_forecast)
            metric_value = np.mean(np.abs(hybrid_forecast[actuals_col].iloc[20:] - hybrid_forecast['hybrid'].iloc[20:])) / np.mean(naive_errors)

        # Store the best weights and metric
        if metric_value < best_metric:
            best_metric = metric_value
            best_weights = (w_prophet, w_lstm)
        print(metric_value)

    return best_weights, best_metric

In [47]:
best_weights, best_metric = find_best_weights(
    hybrid_forecast=hybrid_forecast, 
    actuals_col='y', 
    prophet_col='prophet', 
    lstm_col='lstm', 
    metric='rmse', 
    step=0.1
)

print(f"Best Weights - Prophet: {best_weights[0]}, LSTM: {best_weights[1]}")
print(f"Best RMSE: {best_metric}")

343.4083864191638
346.9847506103604
360.03980608855517
381.60204176081953
410.3324047054664
444.8443367112923
483.90227953721586
526.4955111943276
571.8346136217859
619.3167912791854
668.4855440491887
Best Weights - Prophet: 0.0, LSTM: 1.0
Best RMSE: 343.4083864191638
